In [9]:
from __future__ import print_function
import numpy as np
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.model_selection import train_test_split


e = 0.0001

def SZMO(x_train,y_train,s_train):
    l, m = np.shape(x_train)
    x_train = x_train.T
    b_sign = [0] * l
    for i in range(l):
        if s_train[i] > 0:
            b_sign[i] = 1
        elif s_train[i] == 0:
            b_sign[i] = 0
        else:
            b_sign[i] = -1

    b0 = [0] * l
    b1 = [1] * l
    b2 = [0] * l
    b =  [0] * l
    
#     p_1 = x.transpose(1,0) # transpose twice
#     p_2 = np.dot(x, p_1)
#     p_3 = np.linalg.inv(p_2)
#     p_4 = np.dot(p_1, p_3)
#     p = np.dot(p_4,x)
    #P = X^T * (X * X^T) ^ -1 * X
    p_inv = np.linalg.inv(np.dot(x_train,x_train.T))
    p = np.dot(np.dot(x_train.T,p_inv),x_train)

    I = np.eye(l, dtype=int)
    j = 0

    while np.linalg.norm(np.array(b1)-np.array(b0), ord=2) >= e:
        b0 = b1
        if j == 0:
            m0 = np.dot(p, b2)
        else:
            m0 = np.dot(p, b0)
        m1 = I - p
        m2 = np.dot(m1, y_train)
        m = m0 + m2
        #set-zero-operation
        for i in range(l):
            if b_sign[i] == 1:
                if m[i] <= 0:
                    b[i] = 0
                else:
                    b[i] = m[i]
            elif b_sign[i] == 0:
                b[i] = 0
            else:
                if m[i] >= 0:
                    b[i] = 0
                else:
                    b[i] = m[i]
        b1 = b
        j = j+1
    
    # our method
    w_SZMO = np.dot(np.dot(p_inv,x_train),y_train-b1)
    return w_SZMO

def HARD(x_train,y_train):
    #hard
    l, m = np.shape(x_train)
    k = np.loadtxt('k_hard.txt',dtype = np.float32)
    num = int( k[0] * l)
    x_train = x_train.T
    p_inv = np.linalg.inv(np.dot(x_train,x_train.T))
    p = np.dot(np.dot(x_train.T,p_inv),x_train)
    
    
    b0_h = [0] * l
    b1_h = [1] * l
    b2_h = [0] * l
    
    b0 = np.array(b0_h)
    b_h = np.array(b0_h)
    
    I = np.eye(l, dtype=int)
    j_h = 0
    while np.linalg.norm(np.array(b1_h)-np.array(b0_h), ord=2) >= e:
        b0_h = b1_h
        if j_h == 0:
            m0 = np.dot(p, b2_h)
        else:
            m0 = np.dot(p, b0_h)
        m1 = I - p
        m2 = np.dot(m1, y_train)
        m = m0 + m2
        #print type(b0_h)
        r_h = np.argsort(-b0)
        for i in range(num):
            b_h[r_h[i]] = 0
        b1_h = b_h
        j_h = j_h+1
    #print j_h
    #  hard
    w_hard = np.dot(np.dot(p_inv,x_train),y_train - b1_h)
    return w_hard
def OLS(x_train,y_train):
    x_train = x_train.T
    p_inv = np.linalg.inv(np.dot(x_train,x_train.T))
    
    w_ols = np.dot(np.dot(p_inv,x_train), y_train)
    return w_ols

def main():

    d = 400  # dim of data,m,no-changeable
    n = 3600  # number of sample changeable
    sigma = 1  #changeable
    fold = 5
    
    # load data
    #x samples
    x_data = np.loadtxt('sample.txt',dtype=np.float32)
    x_data = x_data[0:n,0:d]

    #perturbation data 
    #y = y* + \epsilon + b
    y_perturbation_data = np.loadtxt('label_b.txt',dtype = np.float32)
    y_perturbation_data = y_perturbation_data[0:n]
    err = np.random.normal(loc=0, scale=sigma, size=n)
    y_perturbation_data = y_perturbation_data + err
    
    #y = y*
    y_standard_data = np.loadtxt('label.txt',dtype = np.float32)
    y_standard_data = y_standard_data[0:n]
    
    
    
    #sign information of perturbation b
    perturbation = np.loadtxt('s_b.txt',dtype = np.float32)
    perturbation = perturbation[0:n]
    #optimal weight 
    w_optimal = np.loadtxt('w_s.txt',dtype = np.float32)

    y_data = [y_perturbation_data, y_standard_data,perturbation]
    y_data = np.array(y_data)
    y_data = y_data.T

    w_loss = np.zeros([5,6])
    y_loss = np.zeros([5,6])
    
    for ii in range(fold):
        x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.5)
        l, m = np.shape(x_train)
        print("l=", l)
        

        #sign information of perturbation b
        s_train = y_train[:,2]
        #training data label with perturbation b
        y_train= y_train[:,0]
        y_test = y_test[:,1]
        mean = np.mean(y_test)

        # SZMO
        w_szmo = SZMO(x_train,y_train,s_train)
        #print "szmo:{0:.4}".format(np.linalg.norm(w_optimal-w_szmo,ord=2))
        w_loss[ii,0] = np.linalg.norm(w_optimal-w_szmo,ord=2)
        y_predict = np.dot(x_test, w_szmo)
        y_loss[ii,0] = np.linalg.norm(y_predict-y_test,ord=2)/l
        
        #ols
        w_ols = OLS(x_train,y_train)
        #print "ols:{0:.4}".format(np.linalg.norm(w_optimal-w_ols,ord=2))
        w_loss[ii,1] = np.linalg.norm(w_optimal-w_ols,ord=2)
        y_predict = np.dot(x_test, w_ols)
        y_loss[ii,1] = np.linalg.norm(y_predict-y_test,ord=2)/l
        
        
        #HARD
        w_hard = HARD(x_train,y_train)
        #print "hard:{0:.4}".format(np.linalg.norm(w_optimal-w_hard,ord=2))
        w_loss[ii,2] = np.linalg.norm(w_optimal-w_hard,ord=2)
        y_predict = np.dot(x_test, w_hard)
        y_loss[ii,2] = np.linalg.norm(y_predict-y_test,ord=2)/l
        
        # Ridge
        clf = Ridge(alpha=0.3)
        clf.fit(x_train, y_train)
        w_r = clf.coef_
        #print "Ridge:{0:.4}".format(np.linalg.norm(w_optimal-w_r,ord=2))
        w_loss[ii,3] = np.linalg.norm(w_optimal-w_r,ord=2)
        y_predict = np.dot(x_test, w_r)
        y_loss[ii,3] = np.linalg.norm(y_predict-y_test,ord=2)/l
        
        # Lasso
        reg = linear_model.Lasso(alpha=0.1)
        reg.fit(x_train, y_train)
        w_lasso = reg.coef_
        #print "lasso:{0:.4}".format(np.linalg.norm(w_optimal-w_lasso,ord=2))
        w_loss[ii,4] = np.linalg.norm(w_optimal-w_lasso,ord=2)
        y_predict = np.dot(x_test, w_lasso)
        y_loss[ii,4] = np.linalg.norm(y_predict-y_test,ord=2)/l
        
        # Huber
        huber = linear_model.HuberRegressor()
        huber.fit(x_train, y_train)
        w_huber = huber.coef_
        #print "huber:{0:.4}".format(np.linalg.norm(w_optimal-w_huber,ord=2))
        w_loss[ii,5] = np.linalg.norm(w_optimal-w_huber,ord=2)
        y_predict = np.dot(x_test, w_huber)
        y_loss[ii,5] = np.linalg.norm(y_predict-y_test,ord=2)/l

        #     k = np.loadtxt('k_hard.txt',dtype = np.float32)
    #     num = int( k[0] * l)
    #     print("num= ", num)
        #Assign the sign value of perturbation b according to b values.
    #with shape[5,6]
    #w_diff = np.linalg.norm(w_optimal - w_predict,ord = 2,axis = 2)
    mean_w_loss = w_loss.mean(axis=0)
    print ("m:%d,n:%d,sigma:%d,alpha:100"%(d,n/2,sigma))
    #print ("              SZMO    OLS   HARD   LASSO  Ridge Huber")
    #print ("mean_w_loss:",end=' ')
    for i in mean_w_loss:
        print("%.4f" %i,end=' ')
    print("")
    #y_diff = np.linalg.norm(w_optimal - w_predict,ord = 2,axis = 2)
    mean_y_loss = y_loss.mean(axis=0)
    var_y_loss = y_loss.var(axis=0)
    #print ("mean_y_loss:",end=' ')
    for i in mean_y_loss:
        print("%.4f" %i,end=' ')
    print("")
    #print ("var_y_loss :",end=' ')
    var_y_loss = var_y_loss*1000
    for i in var_y_loss:
        print("%.4f" %i,end=' ')
    print("")
    
if __name__ == '__main__':
    main()
    

l= 1800
l= 1800
l= 1800
l= 1800
l= 1800
m:400,n:1800,sigma:1,alpha:100
1.0333 1.2247 1.2247 1.2266 1.2520 1.4642 
0.0739 0.0877 0.0877 0.0986 0.1271 0.1157 
0.0064 0.0048 0.0048 0.1248 0.1608 0.0810 


In [55]:
# y with shape[6,5,2]
import numpy as np
y = [
    [[0.0402 ,0.0706 ,0.0706 ,0.0756 ,0.1732 ,0.0740],[0.0028, 0.0024 ,0.0024 ,0.0314 ,0.3028 ,0.0173]],
    [[0.0471 ,0.0726 ,0.0726 ,0.0745 ,0.1558 ,0.0803],[0.0045 ,0.0064 ,0.0064 ,0.0108 ,0.1526 ,0.0050]],
    [[0.0540 ,0.0794 ,0.0794 ,0.0821 ,0.1542 ,0.0930],[0.0037 ,0.0067 ,0.0067 ,0.0074 ,0.3280 ,0.0340]],
    [[0.0651 ,0.0836 ,0.0836 ,0.0910 ,0.1285 ,0.1081],[0.0051 ,0.0127 ,0.0127 ,0.0328 ,0.1971 ,0.0979]],
    [[0.0728 ,0.0861 ,0.0861 ,0.0882 ,0.1448 ,0.1081],[0.0114 ,0.0134 ,0.0134 ,0.0143 ,0.1305 ,0.0194]]
]




y = np.array(y)
y =y.transpose((2,0,1))

algo = ["OLS  ","HARD ","RIDGE","LASSO","HUBER","SZMO "]

y = y.reshape(6,10)
#print (y)
y[[0,5],:] = y[[5,0],:]
print("m:400,n:1800,sigma:1,alpha:[60,70,80,90,100]")
for i in range(6):
    print(r"\textbf{%s}:"%algo[i],end=' ')
    for j in range(10):
        if j%2==0:
            print("&",end='')
            print("%.4f"%y[i,j],end='$\pm$')
        else:
            print("%.4f"%y[i,j],end='')
    print(r'\\')
#print (y)
#print(y.shape)

m:400,n:1800,sigma:1,alpha:[60,70,80,90,100]
\textbf{OLS  }: &0.0740$\pm$0.0173&0.0803$\pm$0.0050&0.0930$\pm$0.0340&0.1081$\pm$0.0979&0.1081$\pm$0.0194\\
\textbf{HARD }: &0.0706$\pm$0.0024&0.0726$\pm$0.0064&0.0794$\pm$0.0067&0.0836$\pm$0.0127&0.0861$\pm$0.0134\\
\textbf{RIDGE}: &0.0706$\pm$0.0024&0.0726$\pm$0.0064&0.0794$\pm$0.0067&0.0836$\pm$0.0127&0.0861$\pm$0.0134\\
\textbf{LASSO}: &0.0756$\pm$0.0314&0.0745$\pm$0.0108&0.0821$\pm$0.0074&0.0910$\pm$0.0328&0.0882$\pm$0.0143\\
\textbf{HUBER}: &0.1732$\pm$0.3028&0.1558$\pm$0.1526&0.1542$\pm$0.3280&0.1285$\pm$0.1971&0.1448$\pm$0.1305\\
\textbf{SZMO }: &0.0402$\pm$0.0028&0.0471$\pm$0.0045&0.0540$\pm$0.0037&0.0651$\pm$0.0051&0.0728$\pm$0.0114\\


# result of SaPR

##\alpha = [60,70,80,90,100]
m:400,n:1800,sigma:1,alpha:60
0.5791 0.9992 0.9992 1.0008 1.0577 1.0101 
0.0402 0.0706 0.0706 0.0756 0.1732 0.0740 
0.0028 0.0024 0.0024 0.0314 0.3028 0.0173 

m:400,n:1800,sigma:1,alpha:70
0.6589 1.0278 1.0278 1.0286 1.0807 1.1237 
0.0471 0.0726 0.0726 0.0745 0.1558 0.0803 
0.0045 0.0064 0.0064 0.0108 0.1526 0.0050 

m:400,n:1800,sigma:1,alpha:80
0.7654 1.1200 1.1200 1.1209 1.1724 1.2507 
0.0540 0.0794 0.0794 0.0821 0.1542 0.0930 
0.0037 0.0067 0.0067 0.0074 0.3280 0.0340 

m:400,n:1800,sigma:1,alpha:90
0.9116 1.1743 1.1743 1.1775 1.1972 1.3781 
0.0651 0.0836 0.0836 0.0910 0.1285 0.1081 
0.0051 0.0127 0.0127 0.0328 0.1971 0.0979 

m:400,n:1800,sigma:1,alpha:100
1.0204 1.2025 1.2025 1.2028 1.2440 1.4678 
0.0728 0.0861 0.0861 0.0882 0.1448 0.1081 
0.0114 0.0134 0.0134 0.0143 0.1305 0.0194 

In [54]:
y = [
    [[0.0725 ,0.1193 ,0.1193 ,0.1202 ,0.3224 ,0.3647],[0.0134 ,0.0220 ,0.0220 ,0.0291 ,0.2077 ,7.3572]],
    [[0.0934 ,0.1489 ,0.1489 ,0.1683 ,0.4383 ,0.5550],[0.0413 ,0.0337 ,0.0337 ,0.1369 ,1.3775 ,0.0675]],
    [[0.1330 ,0.1923 ,0.1923 ,0.2030 ,0.5648 ,0.6306],[0.0136 ,0.0385 ,0.0385 ,0.1570 ,2.4627 ,0.0637]],
    [[0.1605 ,0.2264 ,0.2264 ,0.2276 ,0.6825 ,0.6922],[0.0177 ,0.0520 ,0.0520 ,0.0518 ,0.3295, 0.1277]],
    [[0.2115 ,0.2851 ,0.2851 ,0.3074 ,0.9202 ,0.7309],[0.0311 ,0.0608 ,0.0608 ,1.0069, 7.1318 ,0.1730]],
]

y = np.array(y)
y =y.transpose((2,0,1))

algo = ["OLS  ","HARD ","RIDGE","LASSO","HUBER","SZMO "]

y = y.reshape(6,10)
#print (y)
y[[0,5],:] = y[[5,0],:]
print("m:[600,700,800,900,1000],n:1800,sigma:1,alpha:50")
for i in range(6):
    print(r"\textbf{%s}:"%algo[i],end=' ')
    for j in range(10):
        if j%2==0:
            print("&",end='')
            print("%.4f"%y[i,j],end='$\pm$')
        else:
            print("%.4f"%y[i,j],end='')
    print(r'\\')

m:[600,700,800,900,1000],n:1800,sigma:1,alpha:50
\textbf{OLS  }: &0.3647$\pm$7.3572&0.5550$\pm$0.0675&0.6306$\pm$0.0637&0.6922$\pm$0.1277&0.7309$\pm$0.1730\\
\textbf{HARD }: &0.1193$\pm$0.0220&0.1489$\pm$0.0337&0.1923$\pm$0.0385&0.2264$\pm$0.0520&0.2851$\pm$0.0608\\
\textbf{RIDGE}: &0.1193$\pm$0.0220&0.1489$\pm$0.0337&0.1923$\pm$0.0385&0.2264$\pm$0.0520&0.2851$\pm$0.0608\\
\textbf{LASSO}: &0.1202$\pm$0.0291&0.1683$\pm$0.1369&0.2030$\pm$0.1570&0.2276$\pm$0.0518&0.3074$\pm$1.0069\\
\textbf{HUBER}: &0.3224$\pm$0.2077&0.4383$\pm$1.3775&0.5648$\pm$2.4627&0.6825$\pm$0.3295&0.9202$\pm$7.1318\\
\textbf{SZMO }: &0.0725$\pm$0.0134&0.0934$\pm$0.0413&0.1330$\pm$0.0136&0.1605$\pm$0.0177&0.2115$\pm$0.0311\\


##m = [600,700,800,900,1000]
m:600,n:1500,sigma:1,alpha:50
0.9221 1.5236 1.5236 1.5231 1.6176 1.7768 
0.0725 0.1193 0.1193 0.1202 0.3224 0.3647 
0.0134 0.0220 0.0220 0.0291 0.2077 7.3572 

1.1912 1.8995 1.8995 1.9043 2.0698 2.2750 
0.0934 0.1489 0.1489 0.1683 0.4383 0.5550 
0.0413 0.0337 0.0337 0.1369 1.3775 0.0675

1.7268 2.5091 2.5091 2.5137 2.7705 3.0142 
0.1330 0.1923 0.1923 0.2030 0.5648 0.6306 
0.0136 0.0385 0.0385 0.1570 2.4627 0.0637 

2.0689 2.9073 2.9073 2.9074 3.2390 3.4855 
0.1605 0.2264 0.2264 0.2276 0.6825 0.6922 
0.0177 0.0520 0.0520 0.0518 0.3295 0.1277

2.7642 3.7370 3.7370 3.7447 4.1530 4.2868 
0.2115 0.2851 0.2851 0.3074 0.9202 0.7309 
0.0311 0.0608 0.0608 1.0069 7.1318 0.1730 

In [53]:
y = [
    [[0.0403 ,0.0768 ,0.0768 ,0.0811 ,0.2074, 0.0762],[0.0010 ,0.0041 ,0.0042 ,0.0062 ,0.0212 ,0.0165]],
    [[0.0481 ,0.0925 ,0.0925 ,0.0945 ,0.2215 ,0.0878],[0.0041 ,0.0123 ,0.0122, 0.0254, 0.1491, 0.0352 ]],
    [[0.0573 ,0.1024 ,0.1024 ,0.1037 ,0.2350 ,0.1142 ],[0.0114, 0.0072, 0.0072, 0.0141, 0.2086, 0.5793 ]],
    [[0.0809 ,0.1292 ,0.1292 ,0.1355 ,0.2961 ,0.2145 ],[ 0.0013 ,0.0214 ,0.0214 ,0.0766 ,1.0795 ,4.9527 ]],
    [[0.1099 ,0.1676 ,0.1676 ,0.1766, 0.4052 ,0.5526],[0.0496 ,0.0384 ,0.0384 ,0.2994 ,2.6414 ,0.9905 ]]
]
y = np.array(y)
y =y.transpose((2,0,1))

algo = ["OLS  ","HARD ","RIDGE","LASSO","HUBER","SZMO "]

y = y.reshape(6,10)
#print (y)
y[[0,5],:] = y[[5,0],:]
print("m:500,n:[1000,1200,1400,1600,1800],sigma:1,alpha:50")
for i in range(6):
    print(r"\textbf{%s}:"%algo[i],end=' ')
    for j in range(10):
        if j%2==0:
            print("&",end='')
            print("%.4f"%y[i,j],end='$\pm$')
        else:
            print("%.4f"%y[i,j],end='')
    print(r'\\')

m:500,n:[1000,1200,1400,1600,1800],sigma:1,alpha:50
\textbf{OLS  }: &0.0762$\pm$0.0165&0.0878$\pm$0.0352&0.1142$\pm$0.5793&0.2145$\pm$4.9527&0.5526$\pm$0.9905\\
\textbf{HARD }: &0.0768$\pm$0.0041&0.0925$\pm$0.0123&0.1024$\pm$0.0072&0.1292$\pm$0.0214&0.1676$\pm$0.0384\\
\textbf{RIDGE}: &0.0768$\pm$0.0042&0.0925$\pm$0.0122&0.1024$\pm$0.0072&0.1292$\pm$0.0214&0.1676$\pm$0.0384\\
\textbf{LASSO}: &0.0811$\pm$0.0062&0.0945$\pm$0.0254&0.1037$\pm$0.0141&0.1355$\pm$0.0766&0.1766$\pm$0.2994\\
\textbf{HUBER}: &0.2074$\pm$0.0212&0.2215$\pm$0.1491&0.2350$\pm$0.2086&0.2961$\pm$1.0795&0.4052$\pm$2.6414\\
\textbf{SZMO }: &0.0403$\pm$0.0010&0.0481$\pm$0.0041&0.0573$\pm$0.0114&0.0809$\pm$0.0013&0.1099$\pm$0.0496\\


##n = [1000,1200,1400,1600,1800]
#n=1800,\sigma=1,\alpha=0.5,m=500,
w:0.5735 ,1.0996 ,1.0996 ,1.1020 ,1.1852 ,1.0197 
y:0.0403 ,0.0768 ,0.0768 ,0.0811 ,0.2074, 0.0762
var:0.0010 ,0.0041 ,0.0042 ,0.0062 ,0.0212 ,0.0165
    
#n=1600,\sigma=1,\alpha=0.5,m=500,
w:0.6480 ,1.2344, 1.2344, 1.2367, 1.3146, 1.1650 
y:0.0481 ,0.0925 ,0.0925 ,0.0945 ,0.2215 ,0.0878 
var:0.0041 ,0.0123 ,0.0122, 0.0254, 0.1491, 0.0352 

#n=1400,\sigma=1,\alpha=0.5,m=500,
w:0.7137, 1.2802, 1.2802, 1.2802, 1.3550, 1.2467 
y:0.0573 ,0.1024 ,0.1024 ,0.1037 ,0.2350 ,0.1142 
var:0.0114, 0.0072, 0.0072, 0.0141, 0.2086, 0.5793 

#n=1200,\sigma=1,\alpha=0.5,m=500,
0.9303, 1.4907, 1.4906, 1.4916, 1.5911, 1.6487 
0.0809 ,0.1292 ,0.1292 ,0.1355 ,0.2961 ,0.2145 
 0.0013 ,0.0214 ,0.0214 ,0.0766 ,1.0795 ,4.9527 

#n=1000,\sigma=1,\alpha=0.5,m=500,
1.1710, 1.7915, 1.7915, 1.7995, 1.9600, 2.2958 
0.1099 ,0.1676 ,0.1676 ,0.1766, 0.4052 ,0.5526 
 0.0496 ,0.0384 ,0.0384 ,0.2994 ,2.6414 ,0.9905 

In [57]:
y = [
    [[0.0413, 0.0631, 0.0631, 0.0655, 0.1559, 0.0642 ],[0.0019, 0.0026, 0.0026, 0.0214, 0.1850, 0.0228  ]],
    [[0.0530, 0.0714, 0.0714, 0.0738, 0.1622, 0.0738],[0.0022, 0.0104, 0.0104, 0.0209, 0.1028, 0.0144]],
    [[ 0.0642, 0.0766, 0.0766, 0.0819, 0.1321, 0.0892 ],[0.0044, 0.0011, 0.0011, 0.0244, 0.2630, 0.0704]],
    [[0.0786, 0.0897, 0.0897, 0.0950, 0.1845, 0.1001],[0.0078, 0.0114, 0.0114, 0.0214, 0.1279, 0.0149]],
    [[0.0881, 0.0933, 0.0933, 0.0944, 0.1580, 0.1018 ],[0.0089, 0.0067, 0.0067, 0.0112, 0.1640, 0.0135 ]]
]
y = np.array(y)
y =y.transpose((2,0,1))

algo = ["OLS  ","HARD ","RIDGE","LASSO","HUBER","SZMO "]

y = y.reshape(6,10)
#print (y)
y[[0,5],:] = y[[5,0],:]
print("m:400,n:1800,sigma:[2,3,4,5,6],alpha:50")
for i in range(6):
    print(r"\textbf{%s}:"%algo[i],end=' ')
    for j in range(10):
        if j%2==0:
            print("&",end='')
            print("%.4f"%y[i,j],end='$\pm$')
        else:
            print("%.4f"%y[i,j],end='')
    print(r'\\')

m:400,n:1800,sigma:[2,3,4,5,6],alpha:50
\textbf{OLS  }: &0.0642$\pm$0.0228&0.0738$\pm$0.0144&0.0892$\pm$0.0704&0.1001$\pm$0.0149&0.1018$\pm$0.0135\\
\textbf{HARD }: &0.0631$\pm$0.0026&0.0714$\pm$0.0104&0.0766$\pm$0.0011&0.0897$\pm$0.0114&0.0933$\pm$0.0067\\
\textbf{RIDGE}: &0.0631$\pm$0.0026&0.0714$\pm$0.0104&0.0766$\pm$0.0011&0.0897$\pm$0.0114&0.0933$\pm$0.0067\\
\textbf{LASSO}: &0.0655$\pm$0.0214&0.0738$\pm$0.0209&0.0819$\pm$0.0244&0.0950$\pm$0.0214&0.0944$\pm$0.0112\\
\textbf{HUBER}: &0.1559$\pm$0.1850&0.1622$\pm$0.1028&0.1321$\pm$0.2630&0.1845$\pm$0.1279&0.1580$\pm$0.1640\\
\textbf{SZMO }: &0.0413$\pm$0.0019&0.0530$\pm$0.0022&0.0642$\pm$0.0044&0.0786$\pm$0.0078&0.0881$\pm$0.0089\\


##\sigma =[2,3,4,5,6]
#\sigma=2,n=1800,\alpha=0.5,m=400
0.5915, 0.8958, 0.8958, 0.8977, 0.9585, 0.8972 
 0.0413, 0.0631, 0.0631, 0.0655, 0.1559, 0.0642 
  0.0019, 0.0026, 0.0026, 0.0214, 0.1850, 0.0228  

0.7504, 1.0192, 1.0192, 1.0208, 1.0815, 1.0504 
 0.0530, 0.0714, 0.0714, 0.0738, 0.1622, 0.0738 
  0.0022, 0.0104, 0.0104, 0.0209, 0.1028, 0.0144 

0.9182, 1.0903, 1.0903, 1.0931 ,1.1181, 1.1645 
 0.0642, 0.0766, 0.0766, 0.0819, 0.1321, 0.0892 
  0.0044, 0.0011, 0.0011, 0.0244, 0.2630, 0.0704 

1.1122, 1.2704, 1.2704, 1.2721, 1.3256, 1.3434 
 0.0786, 0.0897, 0.0897, 0.0950, 0.1845, 0.1001 
  0.0078, 0.0114, 0.0114, 0.0214, 0.1279, 0.0149 

1.2402, 1.3163, 1.3163, 1.3163, 1.3517, 1.4263 
 0.0881, 0.0933, 0.0933, 0.0944, 0.1580, 0.1018 
  0.0089, 0.0067, 0.0067, 0.0112, 0.1640, 0.0135 

In [ ]:
a = [1,2,3]
for i in a:
    print "{},".format(i)

In [55]:
import numpy as np


# generator data and noise
mu, sigma = 1, 3  # gaussion parameter
er = 1
d = 400   # dim of data
n0 = 10000  # number of sample
n1 = 0  #  no noise
n = n1 + n0
k = n0 / n
k0 = np.ones(5) * k
# init parameter, generator w*
y = np.arange(n)
w_s = np.random.rand(d)
# w_s = np.random.normal(loc=3, scale=100, size=d)
nw = np.linalg.norm(w_s)
w_s = w_s/nw * d
err = np.random.normal(loc=0, scale=er, size=n)
# generator matrix X
x = np.random.normal(loc=mu, scale=sigma, size=(n, d))
# generator vector y
y_x = np.dot(x,w_s)
y = np.dot(x,w_s) + err
b = np.ones(n)
s_b = [0] * n
# generator N and P label
s_1 = np.random.rand(n0)-0.5
s_2 = np.zeros(n1)
s = np.hstack((s_1, s_2))
np.random.shuffle(s)  # 打乱s的顺序
print(s)
for i in range(n):
    if s[i] > 0:
        s_b[i] = 1
        b[i] =np.random.rand(1) * nw
    elif s[i]==0:
        b[i] = 0
        s_b[i] = 0
    else:
        b[i] = (np.random.rand(1)-1) * nw
        s_b[i] = -1

y_b = y + b
print(y_b.shape,"yb")
print(y.shape,"y")
print(b.shape,"b")
# save data
np.savetxt('sample.txt', x, fmt='%0.8f')
np.savetxt('label.txt', y_x, fmt='%0.8f')
np.savetxt('label_b.txt', y_b, fmt='%0.8f')
np.savetxt('w_s.txt', w_s, fmt='%0.8f')
np.savetxt('s_b.txt', s_b, fmt='%0.8f')
np.savetxt('k_hard.txt', k0, fmt='%0.8f')

[-0.22987831  0.16673     0.4280553  ... -0.43820838 -0.09504251
 -0.32936957]
(10000,) yb
(10000,) y
(10000,) b


In [ ]:
import numpy as np
err = 1
n = 10000
y_w = np.loadtxt('label_b.txt',dtype = np.float32)
err = np.random.normal(loc=0, scale=err, size=n)
y = y_w + err
np.savetxt('label_b.txt', y, fmt='%0.8f')

In [ ]:
err = 0
n = 5
np.random.normal(loc=0, scale=err, size=n)